In [1]:
import os
from ray.tune.logger import pretty_print
from ray.rllib.algorithms.dqn import DQNConfig, DQN
from ray.rllib.algorithms import Algorithm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

2023-08-17 15:41:39,194	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-08-17 15:41:39,240	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-08-17 15:41:39,242	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!


In [2]:
print_config = False
print_weights = False
print_model = False
checkpoint_dir = "./CartPoleV1/RL_checkpoints"

In [3]:
config = DQNConfig()
config.resources(
    num_gpus=0,
    num_cpus_per_worker=1,
    num_gpus_per_worker=0,
)
config.rollouts(
    num_rollout_workers=1,
    num_envs_per_worker=1,
    create_env_on_local_worker=True,
)
config.environment(
    env="CartPole-v1",
    # env_config={"my_config": "value"},
    render_env=True,
)

In [4]:
algo = config.build()
policy = algo.get_policy()
if print_weights: print(policy.get_weights())

if print_model: print(policy.model)
result = None
for i in range(11):
    print(f"trianing epoch {i}...")
    result = algo.train()
    if i % 5 == 0:
        checkpoint = algo.save(checkpoint_dir)
        print(f"Checkpoint saved to {checkpoint}")

if result is not None:
    print(pretty_print(result))
algo.stop()

2023-08-17 15:41:39,253	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
2023-08-17 15:41:40,750	INFO worker.py:1621 -- Started a local Ray instance.
(pid=37941) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=37941) 2023-08-17 15:41:43,804	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=37941) 2023-08-17 15:41:43,805	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=37941) /home/jim/mambaforge/envs/ray/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(RolloutWorker pid=37941)

trianing epoch 0...


2023-08-17 15:41:46,030	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


Checkpoint saved to ./CartPoleV1/RL_checkpoints/checkpoint_000001
trianing epoch 1...
trianing epoch 2...
trianing epoch 3...
trianing epoch 4...
trianing epoch 5...
Checkpoint saved to ./CartPoleV1/RL_checkpoints/checkpoint_000006
trianing epoch 6...
trianing epoch 7...
trianing epoch 8...
trianing epoch 9...
trianing epoch 10...
Checkpoint saved to ./CartPoleV1/RL_checkpoints/checkpoint_000011
agent_timesteps_total: 11000
connector_metrics:
  ObsPreprocessorConnector_ms: 0.004956483840942383
  StateBufferConnector_ms: 0.0033855438232421875
  ViewRequirementAgentConnector_ms: 0.07801985740661621
counters:
  last_target_update_ts: 10501
  num_agent_steps_sampled: 11000
  num_agent_steps_trained: 320000
  num_env_steps_sampled: 11000
  num_env_steps_trained: 320000
  num_target_updates: 20
custom_metrics: {}
date: 2023-08-17_15-44-06
done: false
episode_len_mean: 77.92
episode_media: {}
episode_reward_max: 279.0
episode_reward_mean: 77.92
episode_reward_min: 11.0
episodes_this_iter: 10


In [5]:
# https://docs.ray.io/en/latest/rllib/rllib-offline.html#off-policy-estimation-ope

restored_algo = Algorithm.from_checkpoint(checkpoint)
evaluation = restored_algo.evaluate()
print(pretty_print(evaluation))

(pid=38226) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=38226) 2023-08-17 15:44:09,491	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=38226) 2023-08-17 15:44:09,492	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-08-17 15:44:09,523	WARNING util.py:68 -- Install gputil for GPU system monitoring.


evaluation:
  connector_metrics: {}
  custom_metrics: {}
  episode_len_mean: .nan
  episode_media: {}
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  hist_stats:
    episode_lengths: []
    episode_reward: []
  num_agent_steps_sampled_this_iter: 10
  num_env_steps_sampled_this_iter: 10
  num_faulty_episodes: 0
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf: {}
  sampler_results:
    connector_metrics: {}
    custom_metrics: {}
    episode_len_mean: .nan
    episode_media: {}
    episode_reward_max: .nan
    episode_reward_mean: .nan
    episode_reward_min: .nan
    episodes_this_iter: 0
    hist_stats:
      episode_lengths: []
      episode_reward: []
    num_faulty_episodes: 0
    policy_reward_max: {}
    policy_reward_mean: {}
    policy_reward_min: {}
    sampler_perf: {}
  timesteps_this_iter: 10



In [6]:
checkpoint

'./CartPoleV1/RL_checkpoints/checkpoint_000011'

In [7]:
# Note: `gymnasium` (not `gym`) will be **the** API supported by RLlib from Ray 2.3 on.
import gymnasium as gym

env_name = "CartPole-v1"
env = gym.make(env_name, render_mode="human")
algo = restored_algo

episode_reward = 0
terminated = truncated = False

obs, info = env.reset()

while not terminated and not truncated:
    action = algo.compute_single_action(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    episode_reward += reward
print(episode_reward)

(RolloutWorker pid=38226) /home/jim/mambaforge/envs/ray/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(RolloutWorker pid=38226)   if not isinstance(terminated, (bool, np.bool8)):
(RolloutWorker pid=38226) 2023-08-17 15:44:09,496	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=38226) 2023-08-17 15:44:09,496	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=38226) 2023-08-17 15:44:09,496	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=38226) 2023-08-

40.0
